<a href="https://colab.research.google.com/github/aubreymoore/crb-damage-detector-colab/blob/main/detect_and_annotate_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# detect_and_annotate.ipynb

NOTE: The following documentation is already slightly out of date.
Please visit https://github.com/aubreymoore/crb-damage-detector-colab before running this notebook for the first time.

This Colab Jupyter notebook runs a custom YOLOv8 object detector which scans images to find three object classes: live coconut palms, dead coconut palms and v-shaped cuts symptomatic of damage caused by coconut rhinoceros beetle, *Oryctes rhinoceros*.

IMPORTANT: Shortly after the MAIN PROGRAM section begins executing, a BROWSE button will appear below the active cell to allow you to upload single file of input data from your loacal machine to Colab.

**Note that Colab will just sit there and not do anything until you have entered a path to a test file of URLs or a ZIP file of images on your local machine.** [Click here to scroll down to the "Browser" button.](#scrollTo=5zSjfTXvIv2q&line=1&uniqifier=1)

You may choose between 2 options:
* A TEXT file (\*.txt) containing URLs for images to be scanned. One URL per line. (This is the most efficient option.)
* A ZIP file (\*.zip) containing images to be scanned.


Test data are available in a companion GitHub repository at https://github.com/aubreymoore/crb-damage-detector-colab. To use the test data, download it to your local computer it as a [ZIP file](https://github.com/aubreymoore/crb-damage-detector-colab/archive/refs/heads/main.zip) and unzip it. If you have **git** installed, you can clone the repo as an alternative. The TEXT file or ZIP file to be uploaded to Colab will be found in the repository's **data** folder.

To scan images, select **Runtime | Run all** on the main menu.
Results will be in a temporary OUTPUT folder which you can access using the **File browser** in the left Colab panel.

When image scanning is complete, the OUTPUT folder will be compressed into a single ZIP file and automatically downloaded to your computer.

### TODO

- [ finished 2024-10-19] Reduce size of images in the companion GH repo to max dimension of 960px
- [ ] Copy current trained model to companion GH repo
- [ ] Copy this Jupyter notebook to companion GH repo
- [ ] Add confidence values to bounding box labels.
- [ ] Add database to OUTPUT folder
- [ ] Extract GPS coordinates from image files
- [ ] Figure out how to use URLs to access images stored on OneDrive (Sharepoint)

# Load Python packages which are not preinstalled by Colab

In [1]:
%pip install ultralytics -q
%pip install supervision -q
# %pip install imutils -q
%pip install icecream -q
%pip install ipython-autotime -q
%pip install exif -q
%pip install requests -q

# Import modules

In [2]:
import cv2
import supervision as sv
from ultralytics import YOLO
# import imutils

import glob
import os
import shutil
# from skimage import io
from icecream import ic
from google.colab import files
import zipfile
# import io
# from urllib.request import urlretrieve

import exif
import numpy as np
import requests

# ultralytics.checks()

# Load cell timer

In [3]:
%load_ext autotime

time: 377 µs (started: 2024-10-25 09:06:25 +00:00)


# Define functions

In [4]:
# url = 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0532.JPG?raw=true'
# filename = 'IMG_0532.JPG'
# urlretrieve(url, filename)

time: 342 µs (started: 2024-10-25 09:06:25 +00:00)


In [5]:
def extract_img_exif(data):
  """
  Extracts an image and EXIF metadata from a given data buffer.

  Returns img as a numpy.ndarray and exif_data as a dict.
  """
  # Extract metadata stored in EXIF
  exif_data = {}
  try:
    exif_data = exif.Image(data).get_all()
  except Exception as e:
    ic(e)
    pass

  # Extract image
  bytes_as_np_array = np.frombuffer(data, dtype=np.uint8)
  img = cv2.imdecode(bytes_as_np_array, cv2.IMREAD_UNCHANGED)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  return img, exif_data

time: 898 µs (started: 2024-10-25 09:06:25 +00:00)


In [6]:
def url2img(url):
  """
  Loads contents of a file referenced by an URL into memory and extracts an
  image and EXIF metadata

  Returns img as a numpy.ndarray and exif_data as a dict.
  """
  # Download the data file referenced by the URL and save contents as "data"
  try:
    response = requests.get(url)
    data = response.content
  except Exception as e:
    ic(e)
    return None, None

  img, exif_data = extract_img_exif(data)
  return img, exif_data

# Usage:
# url = 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0532.JPG?raw=true'
# img, exif_data = url2img(url)
# ic(img)
# ic(exif_data)

time: 772 µs (started: 2024-10-25 09:06:25 +00:00)


In [7]:
def process_zipped_images(zip_file_path):
  """

  Args:
    zip_file_path:
  """
  z = zipfile.ZipFile(zip_file_path)
  ic(z.namelist())
  for file_name in z.namelist():
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):  # Check for common image extensions
      with z.open(file_name, 'r') as file:   # Use z.open to directly open the file within the zip archive
        ic(file_name)
        try:
          data = file.read() # Read the content of the file as bytes
          ic(type(data))
          img, exif_data = extract_img_exif(data)
          ic(img)
          ic(exif_data)

          # Processing code goes here.

        except Exception as e:
          ic(f"Error processing {file_name}: {e}")

# Usage:
# process_zipped_images('images.zip')

time: 1.07 ms (started: 2024-10-25 09:06:25 +00:00)


In [8]:
# def get_gps_from_exif(image_path):
#   """
#   Gets timestamp and GPS coordinates from an image.

#   Args:
#     image_path:

#   Returns:
#     timestamp, latitude, longitude
#   """
#   with open(image_path, 'rb') as src:
#     img = Image(src)
#     if img.has_exif:
#       try:
#         timestamp = img.datetime_original
#         print(img.gps_latitude)
#         dms = img.gps_latitude
#         latitude = dms[0] + dms[1]/60 + dms[2]/3600
#         if img.gps_latitude_ref == 'S':
#           latitude = -latitude
#         dms = img.gps_longitude
#         longitude  = dms[0] + dms[1]/60 + dms[2]/3600
#         if img.gps_longitude_ref == 'W':
#           longitude = -longitude
#         return {"timestamp": timestamp, "latitude": latitude, "longitude": longitude}
#       except Exception as e:
#         print(e)
#         return {"timestamp": None, "latitude": None,"longitude": None}
#     else:
#       print ('The Image has no EXIF')
#       return {"timestamp": None, "latitude": None,"longitude": None}

# # get_gps_from_exif('IMG_0532.JPG')

time: 382 µs (started: 2024-10-25 09:06:25 +00:00)


In [9]:
def get_gps_from_exif(exif_dict):
  """
  Gets timestamp and GPS coordinates for an image.

  Args:
    exif_dict

  Returns:
    timestamp, latitude, longitude
  """
  if exif_dict:
    try:
      timestamp = exif_dict.get('datetime_original', None)
      latitude = exif_dict.get('gps_latitude', None)
      longitude = exif_dict.get('gps_longitude', None)
      if latitude and longitude:
        dms = img.gps_latitude
        latitude = dms[0] + dms[1]/60 + dms[2]/3600
        if img.gps_latitude_ref == 'S':
          latitude = -latitude
          dms = img.gps_longitude
          longitude  = dms[0] + dms[1]/60 + dms[2]/3600
          if img.gps_longitude_ref == 'W':
            longitude = -longitude
      return {"timestamp": timestamp, "latitude": latitude, "longitude": longitude}
    except Exception as e:
      ic(e)
      return {"timestamp": None, "latitude": None,"longitude": None}
    else:
      ic('The Image has no EXIF')
      return {"timestamp": None, "latitude": None,"longitude": None}

# gps_data = get_gps_from_exif(exif_data)

time: 1.89 ms (started: 2024-10-25 09:06:25 +00:00)


In [10]:
def upload_model_weights():
  '''
  Upload model weights from GitHub repo to **weights.pt** only if this file does not already exist.
  '''
  !wget -nc https://github.com/aubreymoore/code-for-CRB-damage-ai/raw/refs/heads/main/models/3class/train5/weights/best.pt -O weights.pt

# upload_model_weights()

time: 992 µs (started: 2024-10-25 09:06:25 +00:00)


In [11]:
def load_model_weights():
  model = YOLO('weights.pt')

time: 539 µs (started: 2024-10-25 09:06:25 +00:00)


In [12]:
def create_input_folder():
  if not os.path.exists('INPUT'):
    os.makedirs('INPUT')

# create_input_folder()

time: 665 µs (started: 2024-10-25 09:06:26 +00:00)


In [13]:
def create_output_folder():
  if not os.path.exists('OUTPUT'):
    os.makedirs('OUTPUT')

# create_output_folder()

time: 4.07 ms (started: 2024-10-25 09:06:26 +00:00)


In [14]:
def run_garbage_disposal():
  '''
  Delete any data files left over from the last run.
  '''
  shutil.rmtree('INPUT', ignore_errors=True)
  shutil.rmtree('OUTPUT', ignore_errors=True)
  shutil.rmtree('sample_data', ignore_errors=True)

  try:
    os.remove('weights.pt')
  except OSError:
    pass

# run_garbage_disposal()

time: 9.01 ms (started: 2024-10-25 09:06:26 +00:00)


In [15]:
def extract_JPG_files(zip_file_path, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Loop through each file in the zip archive
        for file_name in zip_ref.namelist():
            if file_name.endswith('.JPG'):  # Check for .jpg extension
                print(f'Extracting {file_name}...')
                zip_ref.extract(file_name, output_dir)  # Extract the file

# Usage
# extract_dll_files('path/to/your/archive.zip', 'path/to/extract/directory')


time: 2.93 ms (started: 2024-10-25 09:06:26 +00:00)


In [16]:
def extract_zip_to_memory(zip_content):
    """
    Extracts files from a ZIP archive stored in memory.

    :param zip_content: Bytes of the ZIP file.
    :return: A dictionary of filename and file-like objects.
    """
    extracted_files = {}
    with zipfile.ZipFile(io.BytesIO(zip_content)) as z:
        for file_info in z.infolist():
            with z.open(file_info) as file:
                extracted_files[file_info.filename] = file.read()  # Read file content
    return extracted_files

# Usage example
# Assuming 'zip_data' contains the bytes of your ZIP file
# zip_data = ... (load your ZIP data here)
# files = extract_zip_to_memory(zip_data)
# for name, content in files.items():
#     print(f"Extracted {name} with size {len(content)} bytes")


time: 792 µs (started: 2024-10-25 09:06:26 +00:00)


In [17]:
def upload_zip_or_txt():
  '''
  Upload images in a ZIP (*.zip) or list of URLs (*.txt)
  '''
  input_mode = None
  urls = None
  # image_file_dir = None

  # THE FOLLOWING LINE TRIGGERS APPEARANCE OF THE BROWSE BUTTON
  uploaded = files.upload(target_dir='INPUT')
  filename = list(uploaded.keys())[0]

  if filename.endswith('.txt'):
    input_mode = 'text'
    with open(filename, 'r') as f:
      urls = f.read().splitlines()
  elif filename.endswith('.zip'):
    input_mode = 'zip'
  else:
    raise ValueError('INPUT file must be *.txt or *.zip.')
  return input_mode, urls

# Usage:
# input_mode, urls = upload_and_unpack_zip_or_txt()

time: 972 µs (started: 2024-10-25 09:06:26 +00:00)


In [18]:
def get_input_file_list():
  return glob.glob(f'INPUT/**/*', recursive=True)

# get_input_file_list()

time: 593 µs (started: 2024-10-25 09:06:26 +00:00)


In [19]:
def detect_objects(image, model, box_annotator, label_annotator, csv_sink):
  '''
  detect objects in an image
  returns detections and an annotated image
  '''
  results = model(image)[0]
  detections = sv.Detections.from_ultralytics(results)
  # ic(detections)
  annotated_image = box_annotator.annotate(image, detections=detections)
  labels = [f"{model.model.names[class_id]} {confidence:.2f}" for class_id, confidence in zip(detections.class_id, detections.confidence)]
  annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)
  return detections, annotated_image

# csv_sink = sv.CSVSink('detections.csv')
# csv_sink.open()

# upload_model_weights()
# model = YOLO('weights.pt')
# box_annotator = sv.BoxAnnotator()
# label_annotator = sv.LabelAnnotator()

# url = 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/Vanuatu_July_2022_Sulav/resized-images/IMG_0532.JPG?raw=true'
# image = imutils.url_to_image(url)
# detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink)
# ic(detections)
# sv.plot_image(annotated_image)

# custom_data = {'url': url}
# csv_sink.append(detections, custom_data)

# csv_sink.close()

time: 4.76 ms (started: 2024-10-25 09:06:26 +00:00)


# MAIN PROGRAM

In [20]:
# Clear data files from previous run
run_garbage_disposal()

create_input_folder()
create_output_folder()

# Upload images or list of URLs
# THE FOLLOWING LINE TRIGGERS APPEARANCE OF THE BROWSE BUTTON
input_mode, urls = upload_zip_or_txt()

# Upload weights from trained model and load them
upload_model_weights()
model = YOLO('weights.pt')

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
csv_sink = sv.CSVSink('OUTPUT/detections.csv')
csv_sink.open()

Saving urls.txt to INPUT/urls.txt
--2024-10-25 09:07:19--  https://github.com/aubreymoore/code-for-CRB-damage-ai/raw/refs/heads/main/models/3class/train5/weights/best.pt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aubreymoore/code-for-CRB-damage-ai/refs/heads/main/models/3class/train5/weights/best.pt [following]
--2024-10-25 09:07:19--  https://raw.githubusercontent.com/aubreymoore/code-for-CRB-damage-ai/refs/heads/main/models/3class/train5/weights/best.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6269721 (6.0M) [application/octet-stream]
Saving to: ‘weights.pt’

weights.pt          10

## Please click on the Browse buttom when it appears above this cell.

In [21]:
# Scan images
if input_mode == 'text':
  for url in urls:
    ic(url)
    try:
      img, exif_data = url2img(url)
    except Exception as e:
      ic(e)
      continue

    detections, annotated_image = detect_objects(img, model, box_annotator, label_annotator, csv_sink)
    gps_data = get_gps_from_exif(exif_data)
    ic(gps_data)
    csv_sink.append(
        detections,
        custom_data={
            'timestamp': gps_data['timestamp'],
            'latitude': gps_data['latitude'],
            'longitude': gps_data['longitude'],
            'image_h': img.shape[0],
            'image_w': img.shape[1],
            'source': url}
    )
    # Extract filename from URL
    filename = url.split('/')[-1]
    pos = filename.find('?')
    if pos >= 0:
      filename = filename[:pos]

    # Save annotated image
    output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
    ic(output_path)
    os.makedirs(os.path.dirname(output_path), exist_ok = True)
    cv2.imwrite(output_path, annotated_image)

    # try:
    #   # image = imutils.url_to_image(url)
    #   image = cv2.imread('image.jpg')
    #   detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink)
    #   csv_sink.append(
    #       detections,
    #       custom_data={'image_h': image.shape[0], 'image_w': image.shape[1], 'source': url}
    #   )

    #   # Extract filename from URL
    #   filename = url.split('/')[-1]
    #   pos = filename.find('?')
    #   if pos >= 0:
    #     filename = filename[:pos]

    #   output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
    #   ic(output_path)
    #   os.makedirs(os.path.dirname(output_path), exist_ok = True)
    #   cv2.imwrite(output_path, annotated_image)
    # except:
    #   print(f'Error processing {url}')
    # continue

if input_mode == 'zip':
  process_zipped_images('INPUT/images.zip')

  # input_file_list = get_input_file_list()
  # ic(input_file_list)
  # for image_path in input_file_list:
  #   ic(image_path)
  #   try:
  #     image = cv2.imread(image_path)
  #     detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink)
  #     csv_sink.append(
  #         detections,
  #         custom_data={'image_h': image.shape[0], 'image_w': image.shape[1], 'source': image_path}
  #     )

  #     filename = os.path.basename(image_path)
  #     output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
  #     os.makedirs(os.path.dirname(output_path), exist_ok = True)
  #     result = cv2.imwrite(output_path, annotated_image)
  #   except:
  #     print(f'Error processing {image_path}')
  #   continue

csv_sink.close()

ic| url: 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0532.JPG?raw=true'



0: 736x960 11 lives, 383.1ms
Speed: 12.3ms preprocess, 383.1ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 960)


ic| e: AttributeError("'numpy.ndarray' object has no attribute 'gps_latitude'")
ic| gps_data: {'latitude': None, 'longitude': None, 'timestamp': None}
ic| output_path: 'OUTPUT/IMG_0532_annotated.JPG'
ic| url: 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0671.JPG?raw=true'



0: 736x960 3 lives, 369.0ms
Speed: 6.0ms preprocess, 369.0ms inference, 1.0ms postprocess per image at shape (1, 3, 736, 960)


ic| e: AttributeError("'numpy.ndarray' object has no attribute 'gps_latitude'")
ic| gps_data: {'latitude': None, 'longitude': None, 'timestamp': None}
ic| output_path: 'OUTPUT/IMG_0671_annotated.JPG'
ic| url: 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_06XX.JPG?raw=true'
ic| e: error("OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
       ")
ic| url: 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0695.JPG?raw=true'


...


0: 736x960 4 lives, 4 vcuts, 360.5ms
Speed: 5.7ms preprocess, 360.5ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 960)


ic| e: AttributeError("'numpy.ndarray' object has no attribute 'gps_latitude'")
ic| gps_data: {'latitude': None, 'longitude': None, 'timestamp': None}
ic| output_path: 'OUTPUT/IMG_0704_annotated.JPG'
ic| url: 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0713.JPG?raw=true'



0: 736x960 1 live, 339.1ms
Speed: 4.5ms preprocess, 339.1ms inference, 1.0ms postprocess per image at shape (1, 3, 736, 960)


ic| e: AttributeError("'numpy.ndarray' object has no attribute 'gps_latitude'")
ic| gps_data: {'latitude': None, 'longitude': None, 'timestamp': None}
ic| output_path: 'OUTPUT/IMG_0713_annotated.JPG'


time: 7.38 s (started: 2024-10-25 09:07:20 +00:00)


### Download OUTPUT folder as a ZIP file

In [22]:
!zip -r OUTPUT.zip OUTPUT

  adding: OUTPUT/ (stored 0%)
  adding: OUTPUT/IMG_0695_annotated.JPG (deflated 1%)
  adding: OUTPUT/detections.csv (deflated 78%)
  adding: OUTPUT/IMG_0532_annotated.JPG (deflated 0%)
  adding: OUTPUT/IMG_0704_annotated.JPG (deflated 0%)
  adding: OUTPUT/IMG_0671_annotated.JPG (deflated 0%)
  adding: OUTPUT/IMG_0713_annotated.JPG (deflated 1%)
time: 205 ms (started: 2024-10-25 09:07:27 +00:00)


In [23]:
from google.colab import files
files.download("OUTPUT.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 7.22 ms (started: 2024-10-25 09:07:27 +00:00)


# FINISHED
If everything worked as intended, you should find a file named **OUTPUT.zip** in your Downloads folder. Unzip this file to see results.

In [24]:

print('FINISHED')

FINISHED
time: 586 µs (started: 2024-10-25 09:07:28 +00:00)


In [25]:
import zipfile
import io
import PIL
from PIL import Image

z = zipfile.ZipFile('INPUT/images.zip')
for file_name in z.namelist():
  if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):  # Check for common image extensions
    print(f'Extracting {file_name}...')
    with z.open(file_name, 'r') as file: # Use z.open to directly open the file within the zip archive
      try:
        img = Image.open(file) # Pass the file object to Image.open
        # img = Image(file)
        print(img)
        # print(img.gps_latitude)
      # except PIL.UnidentifiedImageError:
      #   print(f"Failed to open {file_name}: UnidentifiedImageError")
      except Exception as e:
        print(f"Failed to open {file_name}: {e}")
      img

FileNotFoundError: [Errno 2] No such file or directory: 'INPUT/images.zip'

time: 31.1 ms (started: 2024-10-25 09:07:28 +00:00)


In [ ]:
img._getexif

